Exploration of the data from the [Di-Tech Challenge](http://research.xiaojukeji.com/competition), organized by Didi Chuxing, a ride-hailing company in China. The data is described [here](http://research.xiaojukeji.com/competition/detail.action?competitionId=DiTech2016).

In [1]:
# Are you running for the final submission?
final_submission = False
# final_submission = True

# Paths
path_to_data = 'data/season_1/'
# test_set_number = '1'
test_set_number = '2'
path_to_test = path_to_data + 'test_set_' + test_set_number + '/'

In [2]:
from time import clock
import glob

import numpy as np
import pandas as pd

# Warn about chained assignment in pandas?
# pd.options.mode.chained_assignment = None

if not final_submission:
    
    # import matplotlib
    # matplotlib.use('Agg')

    from ggplot import *
    %matplotlib inline

/home/vincent/.local/lib/python3.6/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/vincent/.local/lib/python3.6/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/vincent/.local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Order Info Table

<table>
        <tr>
            <th>Field</th>
            <th>Type</th>
            <th>Meaning</th>
            <th>Example</th>
        </tr>
        <tr>
            <td>order_id</td>
            <td>string</td>
            <td>order ID</td>
            <td>70fc7c2bd2caf386bb50f8fd5dfef0cf</td>
        </tr>
        <tr>
            <td>driver_id</td>
            <td>string</td>
            <td>driver ID</td>
            <td>56018323b921dd2c5444f98fb45509de</td>
        </tr>
        <tr>
            <td>passenger_id</td>
            <td>string</td>
            <td>user ID</td>
            <td>238de35f44bbe8a67bdea86a5b0f4719</td>
        </tr>
        <tr>
            <td>start_district_hash</td>
            <td>string</td>
            <td>departure</td>
            <td>d4ec2125aff74eded207d2d915ef682f</td>
        </tr>
        <tr>
            <td>dest_district_hash</td>
            <td>string</td>
            <td>destination</td>
            <td>929ec6c160e6f52c20a4217c7978f681</td>
        </tr>
        <tr>
            <td>Price</td>
            <td>double</td>
            <td>Price</td>
            <td>37.5</td>
        </tr>
        <tr>
            <td>Time</td>
            <td>string</td>
            <td>Timestamp of the order</td>
            <td>2016-01-15 00:35:11</td>
        </tr>
</table>

The Order Info Table shows the basic information of an order, including the passenger and the driver (if driver_id =NULL, it means the order was not answered by any driver), place of origin, destination, price and time. The fields order_id, driver_id, passenger_id, start_hash, and dest_hash are made not sensitive.

In [3]:
def load_orders(order_files):
    """
    Create data frames for order data.
    """
    
    # Columns in order files
    columns = ['order_id', 'driver_id', 'passenger_id', 
               'start_district_hash', 'dest_district_hash', 'price', 'time']

    # Open all of them
    order_dfs = []
    for order_file in order_files:
        order_dfs.append(pd.read_csv(order_file, sep = "\t", names = columns))
    df = pd.concat(order_dfs)

    # Recognize time column as time
    df['time'] = pd.to_datetime(df.time)
    # Set the row labels to the time stamp
    # df = df.set_index('time')

    return df

# Files are organized by dates
if final_submission:
    order_files = glob.glob(path_to_data + 'training_data/order_data/*')
else:
    order_files = [path_to_data + "training_data/order_data/order_data_2016-01-{:02d}".format(i) 
                   for i in range(10, 16)]

df = load_orders(order_files)

# Keep first two weeks for training, next one week for validation.
ind = df['time'] < pd.to_datetime('2016-01-15')
df_train = df[ind]
df_valid = df[~ind]

# Keep a random number of the rows
# df_train = df.sample(frac = 0.70, random_state = 111)
# df_valid = df[~df.index.isin(df_train.index)]

# Avoid looking at validation set during the exploration
df = df_train

# Quick look at the data frame
df.head(2)

,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,price,time
0,8eb94f6832600ee0d342d8a931f64df7,a51979c9a0b257fa81cc401ac8addc97,114cfc989cea164e63d3405fe3055d09,90c5a34f06ac86aee0fd70e2adce7d8a,90c5a34f06ac86aee0fd70e2adce7d8a,7.0,2016-01-10 19:42:02
1,982ca599b83542f32a62fa63fdc25681,9f5decbb0180ab1979ff293bec7d1f93,d52994586c13b6066fd031ba7f684e24,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,8.0,2016-01-10 14:24:23


In [4]:
df.get_dtype_counts()

datetime64[ns]    1
float64           1
object            5
dtype: int64

In [5]:
# Dates?

print("Dates from {} to {}.".format(df['time'].min(), df['time'].max()))

# How many entries? unique orders/passengers/drivers?

num_entries = df.shape[0]
print("{} entries".format(num_entries))

num_orders = len(df.order_id.unique())
print("{} unique orders ({:.1%})".format(num_orders, num_orders/num_entries))

num_pass = len(df.passenger_id.unique())
print("{} unique passengers ({:.1%})".format(num_pass, num_pass/num_entries))
      
num_drivers = len(df.driver_id.unique())
print("{} unique drivers ({:.1%})".format(num_drivers, num_drivers/num_entries))

num_start_district = len(df.start_district_hash.unique())
print("{} unique starting districts ({:.1%})".format(num_start_district, num_start_district/num_entries))

num_dest_district = len(df.dest_district_hash.unique())
print("{} unique destination districts ({:.1%})".format(num_dest_district, num_dest_district/num_entries))

# Price distribution

print("\n")
print(df.price.describe())

Dates from 2016-01-10 00:00:03 to 2016-01-14 23:59:57.
1979690 entries
1975907 unique orders (99.8%)
654563 unique passengers (33.1%)
66048 unique drivers (3.3%)
66 unique starting districts (0.0%)
538 unique destination districts (0.0%)


count    1.979690e+06
mean     1.684720e+01
std      1.598257e+01
min      0.000000e+00
25%      7.900000e+00
50%      1.210000e+01
75%      2.100000e+01
max      5.120000e+02
Name: price, dtype: float64


In [6]:
# Attach time-related info 

def compute_timeinfo(df, dateonly = False):
    # Input: data frame with time (or date) column.
    
    if dateonly:
        # Determine day of week, weekend.
        df['dow'] = pd.to_datetime(df.date).dt.dayofweek
        df['weekend'] = df.dow >= 5
    else:
        # Extract the date, and implicitly make the time midnight.
        df['date'] = df.time.dt.date
        df['timeonly'] = df.time.dt.time
        
        # Compute 10-min timeslots -- absolute and per day.
        df['timeslot_absolute'] = (df['time'] - pd.to_datetime('2016-01-01')).astype('timedelta64[m]')//10
        df['timeslot_day'] = df['timeslot_absolute'] % (24 * 6)
        df['timeslot'] = df['timeslot_day'] % (24*6) + 1
    
        # Determine day of week, weekend.
        df['dow'] = df.time.dt.dayofweek
        df['weekend'] = df.dow >= 5
    
        # Drop the time column?
        # df = df.drop('time', axis = 1)
    
compute_timeinfo(df)

df.head(2)

,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,price,time,date,timeonly,timeslot_absolute,timeslot_day,timeslot,dow,weekend
0,8eb94f6832600ee0d342d8a931f64df7,a51979c9a0b257fa81cc401ac8addc97,114cfc989cea164e63d3405fe3055d09,90c5a34f06ac86aee0fd70e2adce7d8a,90c5a34f06ac86aee0fd70e2adce7d8a,7.0,2016-01-10 19:42:02,2016-01-10,19:42:02,1414.0,118.0,119.0,6,True
1,982ca599b83542f32a62fa63fdc25681,9f5decbb0180ab1979ff293bec7d1f93,d52994586c13b6066fd031ba7f684e24,62afaf3288e236b389af9cfdc5206415,62afaf3288e236b389af9cfdc5206415,8.0,2016-01-10 14:24:23,2016-01-10,14:24:23,1382.0,86.0,87.0,6,True


In [7]:
# Plot of gap vs elapsed time in days group by ten minutes interval

# Convert to days
df['time_from_begin'] = df['timeslot_absolute']/6/24

# Count how many rows per timeslot
# count = df[['order_id', 'driver_id']].groupby('order_id').count()
count = df.groupby('time_from_begin')['time_from_begin'].count()
count = pd.DataFrame(count)
count.columns = ['count']
count.reset_index(level=0, inplace=True)

if not final_submission:
    ggplot(aes('time_from_begin', 'count'), data = count) + \
        geom_point(color = 'gray') + \
        geom_line() + \
        xlab('Elapsed time in days') + ylab('Order count')

In [8]:
# Ranking districts
districts_count = df['start_district_hash'].value_counts(sort = True)

# Number of most popular districts to keep
k = 10
num_entries = df.shape[0]
num_top = districts_count[:10].sum()

# Look at k top districts
print("The first {} most popular districts account for {} out of {} ({:.1%})".format(
        k, num_top, num_entries, num_top/num_entries))

# Rank districts
districts_rank = districts_count.rank(ascending = False, method = 'dense')
districts_top = districts_rank[districts_rank <= k].index

The first 10 most popular districts account for 1327943 out of 1979690 (67.1%)


# Gap

In [9]:
# Count how many rows per order_id and driver_id
# count = df[['order_id', 'driver_id']].groupby('order_id').count()
# count = count.reset_index()
# count = count['driver_id']

# Orders picked up by more than one driver?
# print("Number of orders taken by more than one driver: {}".format(sum(count > 1)))
# Yes..? Surprising.

# Turns out there are duplicate and almost-duplicate entries. 
# The FAQ recommends just leaving them in.

# Remove the duplicates
# dup = df.duplicated(['order_id', 'driver_id', 'passenger_id', 'time'], keep = 'last')
# df = df[~dup]

# Was order answered?
df['is_gap'] = df['driver_id'].isnull()

# Proportion of orders not picked up by a driver
s = sum(df['is_gap'])
l = len(df['is_gap'])
print("There are {} orders-without-drivers out of {} orders: {:.1%}.".format(s, l, s/l))

There are 317114 orders-without-drivers out of 1979690 orders: 16.0%.


In [10]:
# Plot of gap vs elapsed time in days group by ten minutes interval

cols = ['time_from_begin']
df_select = df[cols + ['is_gap']]
df_gap = df_select.groupby(cols).sum()
df_gap = df_gap.reset_index()

if not final_submission:
    ggplot(aes('time_from_begin', 'is_gap'), data = df_gap) + \
        geom_point(color = 'gray') + \
        geom_line() + \
        xlab('Elapsed time in days') + ylab('Gap')

In [11]:
def find_prior_gap(df):
    """
    Find the gap for the three prior time slots. 
    Using shift is fast, but might not lead to the timeslots corresponding to the last 30 min.
    """
    
    # df.sort_values(by = 'timeslot_absolute', inplace = True)
    df.sort_values(by = ['date', 'timeslot'], inplace = True)
    df_grouped = df.groupby(['start_district_hash'])
    
    df['gap_before_1'] = df_grouped.shift(1).gap.fillna(method = 'bfill')
    df['gap_before_2'] = df_grouped.shift(2).gap.fillna(method = 'bfill')
    df['gap_before_3'] = df_grouped.shift(3).gap.fillna(method = 'bfill')

In [12]:
# Make new data frame with gap per timeslot per district

def compute_gap(df):
    
    # Compute gap column
    # df['gap'] = df.groupby(cols).driver_id.isnull().transform('sum')
    
    # Gap is when no driver
    df['is_gap'] = df['driver_id'].isnull()
    
    # Aggregate
    cols = ['start_district_hash', 'date', 'timeslot']
    df_select = df[cols + ['is_gap']]
    df_gap = df_select.groupby(cols).sum()
    
    # Flatten data frame
    df_gap = df_gap.reset_index()
    df_gap = df_gap.rename(columns = {'is_gap': 'gap'})
    
    # Add prior gap columns
    find_prior_gap(df_gap)
    
    return df_gap

# Compute gap. Only care about new data frame now.
df = compute_gap(df)

# Sanity check: do the number add up?
print(sum(df.gap))

317114.0


# Weather

<table>
        <tr>
            <th>Field</th>
            <th>Type</th>
            <th>Meaning</th>
            <th>Example</th>
        </tr>
        <tr>
            <td>Time</td>
            <td>string</td>
            <td>Timestamp</td>
            <td>2016-01-15 00:35:11</td>
        </tr>
        <tr>
            <td>Weather</td>
            <td>int</td>
            <td>Weather</td>
            <td>7</td>
        </tr>
        <tr>
            <td>temperature</td>
            <td>double</td>
            <td>Temperature</td>
            <td>-9</td>
        </tr>
        <tr>
            <td>PM2.5</td>
            <td>double</td>
            <td>pm25</td>
            <td>66</td>
        </tr>
</table>

The Weather Info Table shows the weather info every 10 minutes each city. The weather field gives the weather conditions such as sunny, rainy, and snowy etc; all sensitive information has been removed. The unit of temperature is Celsius degree, and PM2.5 is the level of air pollutions.

In [13]:
def load_weather(weather_files):

    # Open all of them
    columns = ['time', 'weather', 'temperature', 'pm25']
    weather_dfs = []
    for f in weather_files:
        weather_dfs.append(pd.read_csv(f, sep = "\t", names = columns))
    dfw = pd.concat(weather_dfs)

    # Extract date and time slot
    dfw['time'] = pd.to_datetime(dfw.time)
    dfw['date'] = dfw.time.dt.date
    dfw['timeslot'] = (dfw['time'] - pd.to_datetime(dfw['date'])).astype('timedelta64[m]')//10 + 1
    dfw = dfw.drop('time', axis = 1)

    return dfw

# Files are organized by dates
weather_files = glob.glob(path_to_data + 'training_data/weather_data/*')
dfw = load_weather(weather_files)
# dfw.describe()

In [14]:
def merge_weather(df, dfw):
    # Merge into main data frame, and fill missing values
    # http://pandas.pydata.org/pandas-docs/stable/missing_data.html
    
    df = df.merge(dfw, on = ['date', 'timeslot'], how = 'left')
    
    # Fill missing values with forward/backward fills
    df.sort_values(by = ['date', 'timeslot'], inplace = True)
    df.temperature = df.temperature.fillna(method = 'ffill')
    df.temperature = df.temperature.fillna(method = 'bfill')
    
    # ... or interpolation
    # df.temperature = df.temperature.interpolate(method = 'time')
    
    return df

In [15]:
df = merge_weather(df, dfw)

# Categorical Variables

In [16]:
# Create conversion table
districts_convert = pd.DataFrame(data = districts_rank)
districts_convert.reset_index(level = 0, inplace = True)
districts_convert.columns = ['start_district_hash', 'start_district_rank']
# districts_convert.head(2)

# Replace hash by rank
df = df.merge(districts_convert, on = 'start_district_hash', how = 'left')
# df = df.drop('start_district_hash', axis = 1)

In [17]:
def one_hot_encode(df):
    """
    One-hot encode categorical column for most-popular district hash.
    """
    
    # Look only at most popular districts
    df['district'] = np.nan
    df.loc[df.start_district_hash.isin(districts_top), 'district'] = \
        df.loc[df.start_district_hash.isin(districts_top), 'start_district_hash']
    
    # One-hot encoding
    dummies = pd.get_dummies(df['district'], dummy_na = False)
    df = pd.concat((df.drop('district', axis = 1), dummies.astype(int)), axis = 1)
    
    return df

# df = one_hot_encode(df)

# District Info Table

<table>
        <tr>
            <th>Field</th>
            <th>Type</th>
            <th>Meaning</th>
            <th>Example</th>
        </tr>
        <tr>
            <td>district_hash</td>
            <td>string</td>
            <td>District hash</td>
            <td>90c5a34f06ac86aee0fd70e2adce7d8a</td>
        </tr>
        <tr>
            <td>district_id</td>
            <td>string</td>
            <td>District ID</td>
            <td>1</td>
        </tr>
</table>

The District Info Table shows the information about the districts to be evaluated in the contest. You need to do the prediction given the districts from the District Definition Table. In the submission of the results, you need to map the district hash value to district mapped ID.

In [18]:
# Use the starting district_hash as the associated disctrict
col = 'start_district_hash'

# Load district conversion table
district_file = path_to_data + 'training_data/cluster_map/cluster_map'
district = pd.read_csv(district_file, sep = '\t', names = [col, 'district_id'])

# How many districts?
print("There are {} districts in the district file.".format(district.shape[0]))

There are 66 districts in the district file.


# Test Data

In [19]:
# Columns in order files
columns = ['datetimeslot']

# Open list of slots
order_file = path_to_test + 'read_me_' + test_set_number + '.txt'
df_test_slots = pd.read_csv(order_file, sep = "\t", names = columns, skiprows = 1)

# Extract date and timeslot [0,143]
df_test_slots['date'] = pd.to_datetime(df_test_slots.datetimeslot.str[:10])
df_test_slots['timeslot'] = df_test_slots.datetimeslot.str[11:]
df_test_slots['timeslot'] = df_test_slots.timeslot.astype(int)
compute_timeinfo(df_test_slots, dateonly = True)

# Create all combination of districts and time slots
df_test_slots['to_predict'] = True
district['to_predict'] = True
df_test_slots = df_test_slots.merge(district, on = 'to_predict')

In [20]:
# Load test order files
order_files = glob.glob(path_to_test + 'order_data/*')
df_test = load_orders(order_files)

# Attach time info
compute_timeinfo(df_test)

# Compute gap per time slot per district
df_test = compute_gap(df_test)

In [21]:
# Merge with slots to predict in test set
df_test['to_predict'] = False
df_test_full = pd.concat([df_test, df_test_slots])

In [22]:
# Load weather data
order_files = glob.glob(path_to_test + 'weather_data/*')
dfw_test = load_weather(weather_files)

# Get all the weather data
dfw_all = pd.concat([dfw, dfw_test])

# Merge weather data
df_test_full = merge_weather(df_test_full, dfw_all)

# Replace district by popularity
df_test_full = df_test_full.merge(districts_convert, on = 'start_district_hash', how = 'left')

In [23]:
df_test_full.head(5)

,date,datetimeslot,district_id,dow,gap,gap_before_1,gap_before_2,gap_before_3,start_district_hash,timeslot,to_predict,weekend,weather,temperature,pm25,start_district_rank
0,2016-01-23,NaN,NaN,NaN,37.0,37.0,37.0,37.0,1afd7afbc81ecc1b13886a569d869e8a,43.0,False,NaN,NaN,NaN,NaN,9.0
1,2016-01-23,NaN,NaN,NaN,2.0,37.0,37.0,37.0,1c60154546102e6525f68cb4f31e0657,43.0,False,NaN,NaN,NaN,NaN,46.0
2,2016-01-23,NaN,NaN,NaN,5.0,37.0,37.0,37.0,1cbfbdd079ef93e74405c53fcfff8567,43.0,False,NaN,NaN,NaN,NaN,27.0
3,2016-01-23,NaN,NaN,NaN,0.0,37.0,37.0,37.0,1ecbb52d73c522f184a6fc53128b1ea1,43.0,False,NaN,NaN,NaN,NaN,36.0
4,2016-01-23,NaN,NaN,NaN,3.0,37.0,37.0,37.0,2301bc920194c95cf0c7486e5675243c,43.0,False,NaN,NaN,NaN,NaN,32.0


In [24]:
# Compute prior gap, and then keep only the data we need to predict
find_prior_gap(df_test_full)

# Re-divide the two data frames
df_test = df_test[~df_test_full['to_predict']]
df_test_slots = df_test_full[df_test_full['to_predict']]

# Drop extra columns
df_test.drop('to_predict', axis = 1, inplace = True)
df_test_slots.drop('to_predict', axis = 1, inplace = True)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


# Preparation for Validation

In [25]:
# Compute timeslot
compute_timeinfo(df_valid)

# Compute gap per time slot per district
df_valid = compute_gap(df_valid)

# Merge temperature
df_valid = merge_weather(df_valid, dfw)

# One-hot encoding of districts
# df_valid = one_hot_encode(df_valid)

# Replace district by popularity
df_valid = df_valid.merge(districts_convert, on = 'start_district_hash', how = 'left')

# Extract timeslots for testing in validation
timeslots = df_test_slots.timeslot.unique()
df_valid = df_valid[df_valid['timeslot'].isin(timeslots)]

# Check data frame out
df_valid.head(5)

,start_district_hash,date,timeslot,gap,gap_before_1,gap_before_2,gap_before_3,weather,temperature,pm25,start_district_rank
3167,08232402614a9b48895cc3d0aeb0e9f2,2016-01-15,46.0,0.0,2.0,1.0,1.0,2.0,4.0,154.0,60.0
3168,08f5b445ec6b29deba62e6fd8b0325a6,2016-01-15,46.0,0.0,0.0,0.0,0.0,2.0,4.0,154.0,59.0
3169,1afd7afbc81ecc1b13886a569d869e8a,2016-01-15,46.0,33.0,5.0,4.0,2.0,2.0,4.0,154.0,9.0
3170,1c60154546102e6525f68cb4f31e0657,2016-01-15,46.0,1.0,0.0,1.0,0.0,2.0,4.0,154.0,46.0
3171,1cbfbdd079ef93e74405c53fcfff8567,2016-01-15,46.0,1.0,2.0,0.0,1.0,2.0,4.0,154.0,27.0


# Preparation for Final Training

In [26]:
# Is it a weekend or weekday?
compute_timeinfo(df, dateonly = True)
compute_timeinfo(df_valid, dateonly = True)
compute_timeinfo(df_test, dateonly = True)

# Concatenate
df_all = pd.concat([df, df_valid, df_test])

# Predictions by Clusters

In [27]:
class predict_by_cluster():
    # Make predictions by simply taking the median per start_district_hash per timeslot per weekend
    
    # Select columns
    cols = ['start_district_hash', 'weekend', 'timeslot']
    # cols = ['start_district_hash', 'dow', 'timeslot']
    
    def __init__(self, df):
        
        # Make prediction data frame
        df = df[self.cols + ['gap']]
        df_grouped = df.groupby(self.cols)
        gap_cluster = df_grouped.median() # median, mean, min, max
        gap_cluster = gap_cluster.reset_index().rename(columns = {'gap': 'gap_cluster'})
        
        self.gap_cluster = gap_cluster

    def merge(self, df):
        
        # Merge gap_cluster in data frame
        df = df.merge(self.gap_cluster, on = self.cols, how = 'left')

        # Fill missing predictions
        df['gap_cluster'] = df['gap_cluster'].fillna(method = 'ffill')
        df['gap_cluster'] = df['gap_cluster'].fillna(method = 'bfill')
        
        return df

# Make predictions
gap_cluster = predict_by_cluster(df)

# Make prediction table per group with ALL the data -- for final submission
gap_cluster_all = predict_by_cluster(df_all)

In [28]:
# Merge back into main data frame
df = gap_cluster.merge(df)

# Set given outcome and predictions
train_outcome = df['gap']
train_predict = df['gap_cluster']

# Merge back into main data frame
df_valid = gap_cluster.merge(df_valid)

# Set given outcome and predictions
valid_outcome = df['gap']
valid_predict = df['gap_cluster']

# Set predictions on test slots
df_test_slots = gap_cluster_all.merge(df_test_slots)

# Prediction with sklearn

In [29]:
# Select features
cols = ['start_district_rank', 'dow', 'timeslot', 'temperature', 'gap_before_1', 'gap_before_2', 'gap_before_3']    
train = df_all[cols] if final_submission else df[cols]
# train.head(5)

# Fill missing values
from sklearn.preprocessing import Imputer
imputer = Imputer()
train = imputer.fit_transform(train)

# Reference outcome
train_outcome = df_all['gap'] if final_submission else df['gap']
# train_outcome.head(5)

# print(pd.isnull(train).any(1).nonzero()[0])
# print(train[pd.isnull(train).any(1)])

In [30]:
# Select regressor
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators = 10)
# from sklearn.tree import DecisionTreeRegressor
# reg = DecisionTreeRegressor(max_depth = 3)

# Fit training data
start = clock()
reg.fit(train, train_outcome)
print("Fit in {:.0f} seconds.".format(clock() - start))

# Variable importance
# print(pd.DataFrame(data = [reg.feature_importances_], columns = train.columns, index = ['importance']))

Fit in 2 seconds.


In [31]:
df_test_slots.head(5)

,date,datetimeslot,district_id,dow,gap,gap_before_1,gap_before_2,gap_before_3,start_district_hash,timeslot,weekend,weather,temperature,pm25,start_district_rank,gap_cluster
0,2016-01-23 00:00:00,2016-01-23-46,1.0,5.0,NaN,6.0,11.0,9.0,90c5a34f06ac86aee0fd70e2adce7d8a,46.0,True,NaN,NaN,NaN,6.0,3.0
1,2016-01-23 00:00:00,2016-01-23-46,2.0,5.0,NaN,1.0,0.0,0.0,f2c8c4bb99e6377d21de71275afd6cd2,46.0,True,NaN,NaN,NaN,26.0,0.0
2,2016-01-23 00:00:00,2016-01-23-46,3.0,5.0,NaN,0.0,2.0,16.0,58c7a4888306d8ff3a641d1c0feccbe3,46.0,True,NaN,NaN,NaN,51.0,0.0
3,2016-01-23 00:00:00,2016-01-23-46,4.0,5.0,NaN,4.0,8.0,16.0,b26a240205c852804ff8758628c0a86a,46.0,True,NaN,NaN,NaN,17.0,8.0
4,2016-01-23 00:00:00,2016-01-23-46,5.0,5.0,NaN,2.0,1.0,5.0,4b9e4cf2fbdc8281b8a1f9f12b80ce4d,46.0,True,NaN,NaN,NaN,49.0,0.0


In [32]:
# Extrapolate
train_predict = reg.predict(train)

# Validation
valid_outcome = df_valid['gap']
valid_predict = reg.predict(df_valid[cols])

# Set predictions on test slots
df_test_slots_imputed = imputer.transform(df_test_slots[cols])
# df_test_slots_imputed = df_test_slots[cols]
df_test_slots['gap_predict'] = reg.predict(df_test_slots_imputed)

# Evaluation
Consider di districts and tj time slots, and the supply-demand gap gapij , and your prediction is sij, we use as the evaluation metrics: 
![MAPE](figures/mape.jpg)
The lowest MAPE will be the best.

In [33]:
def mape(outcome, predict):
    """
    Compute Mean Absolute Percentage Error (MAPE) score. Lower is better.
    """
    
    # Verify that there are no missing values
    try:
        assert not (outcome.isnull().values.any() or predict.isnull().values.any())
    except AttributeError:
        assert not (np.isnan(np.sum(outcome)) or np.isnan(np.sum(outcome)))
    
    # Get only the NONZERO elements
    EPSILON = pow(10, -5)
    idx = np.abs(outcome) > EPSILON
    
    # Extract those elements
    outcome = np.array(outcome)[np.where(idx)]
    predict = np.array(predict)[np.where(idx)]
    
    return np.mean(np.abs((outcome - predict) / outcome))

score = mape(train_outcome, train_predict)
print("Training MAPE: {:.6f}".format(score))

score = mape(valid_outcome, valid_predict)
print("Validation MAPE: {:.6f}".format(score))

# The mean by cluster did #516 with 0.428684 on 2016-06-17.

Training MAPE: 0.160941
Validation MAPE: 0.953641


# Output
<table class="table table-2">
        <tr>
            <th>Data name</th>
            <th>Data type</th>
            <th>Example</th>
        </tr>
        <tr>
            <td>District ID</td>
            <td>string</td>
            <td>1,2,3,4 (the same as district mapping ID)</td>
        </tr>
        <tr>
            <td>Time slot</td>
            <td>string</td>
            <td>2016-01-23-1 (The first time slot on Jan. 23rd, 2016; one day is uniformly divided into 144 ten minute time slots)</td>
        </tr>
        <tr>
            <td>Prediction value</td>
            <td>double</td>
            <td>6.0</td>
        </tr>

</table>

In [34]:
# Make the date - timeslot column
df_test_slots['datetimeslot'] = df_test_slots.date.dt.date.map(str) + '-' + df_test_slots.timeslot.astype(int).map(str)

# Prepare output file
cols = ['district_id', 'datetimeslot', 'gap_predict']
df_test_slots[cols].to_csv('predict.csv', index = False, header = False)

# Correlation with other result?

In [ ]:
# Columns in order files
cols = ['district_id', 'datetimeslot', 'gap_predict']

# Open list of slots
order_file = 'predict_final.csv'
results_prior = pd.read_csv(order_file, sep = ",", names = cols)

# Compute correlation
results = results_prior.merge(df_test_slots[cols], on = ['district_id', 'datetimeslot'])
corr = results[['gap_predict_x', 'gap_predict_y']].corr()
corr = corr.iloc[0,1]

print("The correlation between the two results is {:.6f}".format(corr))